<a href="https://colab.research.google.com/github/erezimm/ML-ML/blob/master/GNN_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Main notebook, to be run on Google Colab

## Import dependencies and download datasets

In [2]:
!pip install dgl

from datetime import date
import dgl
import dgl.function as fn
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph
from torch.utils.data import Dataset, DataLoader
import glob
import json
from tqdm.notebook import tqdm
import torch.optim as optim
import numpy as np
import networkx as nx
import astropy.io.ascii
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt

!wget -O data.zip https://github.com/erezimm/ML-ML/raw/master/data.zip
!unzip -o -q data.zip -d ./data
training_path = '/content/data/training_set'
validation_path = '/content/data/validation_set'

  Using cached https://files.pythonhosted.org/packages/c5/b4/84e4ebd70ef3985181ef5d2d2a366a45af0e3cd18d249fb212ac03f683cf/dgl-0.4.3.post2-cp36-cp36m-manylinux1_x86_64.whl


DGL backend not selected or invalid.  Assuming PyTorch for now.
Using backend: pytorch


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)
--2020-08-05 11:38:22--  https://github.com/erezimm/ML-ML/raw/master/data.zip
Resolving github.com (github.com)... 140.82.118.3
Connecting to github.com (github.com)|140.82.118.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/erezimm/ML-ML/master/data.zip [following]
--2020-08-05 11:38:22--  https://raw.githubusercontent.com/erezimm/ML-ML/master/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59777 (58K) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]  

## Class CustomDataset loads all files and converts them to a list of graphs

Each node in a graph is a data point, i.e. node features are days since start, mag, magerr

In [3]:
class CustomDataset(Dataset):
    def __init__(self, path):
        filelist = glob.glob(path+'/*')
        
        self.graphs = []
        self.isMicrolensed = []
        for fname in tqdm(filelist):
            if fname.startswith('clean')
                microlensed = False
            elif fname.startswith('microlensed'):
                microlensed = True
            else:
                raise Exception('Filename ' + fname + ' not clean_*/microlensed_*')
            
            with open(fname) as f:
                data = astropy.io.ascii.read(f.read())
                times, mags, magerrs = data['col1'], data['col2'], data['col3']
                zipped = list(zip(times, mags, magerrs))
                zipped.sort(key=lambda tup: tup[0])
                times, mags, magerrs = zip(*zipped)

                n = len(times)
                g = dgl.DGLGraph()
                g.add_nodes(n)
                g.ndata['time'] = torch.tensor(times)  # days
                g.ndata['mag'] = torch.tensor(mags)
                g.ndata['magerr'] = torch.tensor(magerrs)
                g.add_edges([i for i in range(n)], [(i+1)%n for i in range(n)])
                g.add_edges([i for i in range(n)], [(i-1)%n for i in range(n)])
                
                self.graphs.append(g)
                self.isMicrolensed.append(microlensed)
        
    def __len__(self):
        return len(self.graphs)

    def __getitem__(self, idx):
        return self.graphs[idx], self.isMicrolensed[idx]


train_ds = CustomDataset(training_path)
# validation_ds = CustomDataset(validation_path)

nx.draw(dgl.to_networkx(train_ds[20][0]))

SyntaxError: ignored

In [ ]:
def collate(samples):
    # The input `samples` is a list, a batch of whatever comes out of your dataset object
    graphs = [x[0] for x in samples]
    labels = [x[1] for x in samples]
    
    batched_graph = dgl.batch(graphs,node_attrs=['time', 'mag', 'magerr'])
    targets = torch.cat(labels)
    
    return batched_graph, targets.unsqueeze(1).float()

## The Newtwork

In [ ]:
n_ft = 3  # number of node features (time, mag, magerr)

class EdgeNetwork(nn.Module):
    def __init__(self, node_hidrep, edge_hidrep):
        super(EdgeNetwork, self).__init__()
        n_in = 2*(n_ft + node_hidrep) + edge_hidrep
        
        # params:
        sizes = [n_in] + [] + [edge_hidrep]  # edit middle list, can be empty

        # create networks:
        layers = []
        for i in range(len(sizes)-1):
            layers.extend([nn.Linear(sizes[i], sizes[i+1]), nn.ReLU()])
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        
        #your input x is an object with the following properties:
        # x.dst['node_features'], x.dst['node_hidden_state']
        # x.src['node_features'], x.src['node_hidden_state']
        # x.data['distance']
        
        #put them together with torch.cat

        # print( x.dst['node_features'].shape, x.dst['node_hidden_state'].shape,
        #                     x.src['node_features'].shape, x.src['node_hidden_state'].shape,
        #                     x.data['distance'].shape)

        input = torch.cat(( x.dst['time'], x.dst['mag'], x.dst['magerr'], x.dst['node_hidrep'],
                            x.src['time'], x.src['mag'], x.src['magerr'], x.src['node_hidrep'],
                            x.data['edge_hidrep'] ), dim=1)
        
        #use a neural network to create an edge hidden represetation - 

        output = self.network(input)

        #you return a dictionary with what you want to "send" to the reciving node
        return {'edge_hidrep': output }

    
class NodeNetwork(nn.Module):
    def __init__(self, node_hidrep, edge_hidrep):
        super(NodeNetwork, self).__init__()
        n_in = 2*edge_hidrep + node_hidrep + n_ft

        # params:
        sizes = [n_in] + [] + [node_hidrep]  # edit middle list, can be empty

        # create networks:
        layers = []
        for i in range(len(sizes)-1):
            layers.extend([nn.Linear(sizes[i], sizes[i+1]), nn.ReLU()])
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        
        #this time your input x has:
        # x.mailbox['edge hidden represetation'] -> this is what you send with the edge update function above - 
        # it will have the size of the node neighborhood - 
        # (Batch size, number of nodes in neighborhood, edge hidden rep size), so you need to sum/mean over dim=1 
        # x.data['node_hidden_state'] and x.data['node_features'] (this is the existing state of your node)
        # you need to torch.cat the message sum, node hidden state, and node features 
        #- and then apply some fully connected neural network
        
        input = torch.mean( x.mailbox['edge_hidrep'] , dim=1 )

        # print( mb.shape, x.data['node_hidden_state'].shape, x.data['node_features'].shape )

        input = torch.cat((mb, x.data['node_hidrep'], x.data['time'], x.data['mag'], x.data['magerr']), dim=1)
        out = self.network(input)

        # return a new hidden state for the node
        return {'node_hidrep': out }


class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()

        # params:
        node_hidrep, edge_hidrep = 10, 10
        sizes_nodeinit = [n_ft] + [] + [node_hidrep]  # edit middle list, can be empty
        self.update_loop_length = 20

        # create networks:
        layers = []
        for i in range(len(sizes_nodeinit)-1):
            layers.extend([nn.Linear(sizes_nodeinit[i], sizes_nodeinit[i+1]), nn.ReLU()])
        self.node_init = nn.Sequential(*layers)
        self.edge_network = EdgeNetwork(node_hidrep, edge_hidrep)
        self.node_network = NodeNetwork(node_hidrep, edge_hidrep)
        
    def forward(self, g):
        g.ndata['node_hidrep'] = self.node_init(torch.cat((g.ndata['time'], g.ndata['mag'], g.ndata['magerr'])))
        for i in range(self.update_loop_length):
            g.update_all(self.edge_network, self.node_network)
        
        out = 0
        for x in g.ndata['node_hidrep']:
            out = max(out, torch.sum(x))
        return out 

In [ ]:
if torch.cuda.is_available():
    print('Cuda on')
    device = torch.device('cuda')
else:
  print('No Cuda')
  device = torch.device('cpu')

net = Classifier()
net.to(device)

## Train

In [ ]:
loss_func = nn.BCEWithLogitsLoss()
# optimizer = optim.SGD(net.parameters(), lr=0.01)
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [ ]:
data_loader = DataLoader(train_ds, batch_size=30, shuffle=True, collate_fn=collate)

n_epochs = 50
for epoch in tqdm(range(n_epochs)):
    net.train()
    losses = []
    for x,y in data_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        output = net(x)
        # print(output.shape, y.shape)
        loss = loss_func(output,y)
        losses.append(float(loss))
        loss.backward()
        optimizer.step()
    
    print(epoch, ':', np.mean(np.array(losses)))      

## Evaluate

Once you train your network you can evaluate on the test dataset, and compute the rate of false positive, false negative, etc,

train until you reach at least 60% true positive rate while maintaining a 90%+ true negative rate, and submit

In [ ]:
test_data_loader = DataLoader(test_ds, batch_size=30, shuffle=True, collate_fn=collate)

net.cpu()
net.eval()

true_positive = 0
false_positive = 0
true_negative = 0
false_negative = 0
total_positives = 0
total_negatives = 0

for i, (x,y) in enumerate(test_data_loader):
        
        y = y.data.numpy()

        prediction = net(x).cpu().data.numpy()
        
        prediction[prediction >= 0.5] = 1
        prediction[prediction <0.5] = 0
        
        total_positives+=len(np.where( y==1 )[0])
        total_negatives+=len(np.where( y==0 )[0])
        
        true_positive+= len(np.where( (prediction==y) & (y==1) )[0])
        true_negative+= len(np.where( (prediction==y) & (y==0) )[0])
        false_positive+= len(np.where( (prediction!=y) & (y==0) )[0])
        false_negative+= len(np.where( (prediction!=y) & (y==1) )[0])
        
        
print('true positive: ', true_positive/total_positives)
print(false_negative/total_positives)
print('true negative: ', true_negative/total_negatives)
print(false_positive/total_negatives)

## Save
Don't forget to download

In [ ]:
torch.save(net.state_dict(), 'ML-ML/models/model.pt')